In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("comm") \
        .getOrCreate()

In [55]:
df = spark.read.csv("training.1600000.processed.noemoticon.csv")

In [56]:
df.count()

1600000

In [57]:
df.show(5)

+---+----------+--------------------+--------+---------------+--------------------+
|_c0|       _c1|                 _c2|     _c3|            _c4|                 _c5|
+---+----------+--------------------+--------+---------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|  0|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|  0|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|  0|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|  0|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
+---+----------+--------------------+--------+---------------+--------------------+
only showing top 5 rows



In [58]:
df = df.withColumnRenamed("_c0", "label")\
 .withColumnRenamed("_c1", "id")\
 .withColumnRenamed("_c2", "time")\
 .withColumnRenamed("_c3", "Query_Boolean")\
 .withColumnRenamed("_c4", "Username")\
 .withColumnRenamed("_c5", "Content")

In [59]:
df.show(2)

+-----+----------+--------------------+-------------+---------------+--------------------+
|label|        id|                time|Query_Boolean|       Username|             Content|
+-----+----------+--------------------+-------------+---------------+--------------------+
|    0|1467810369|Mon Apr 06 22:19:...|     NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|    0|1467810672|Mon Apr 06 22:19:...|     NO_QUERY|  scotthamilton|is upset that he ...|
+-----+----------+--------------------+-------------+---------------+--------------------+
only showing top 2 rows



In [60]:
df.filter(df.label == 0).show(10)

+-----+----------+--------------------+-------------+---------------+--------------------+
|label|        id|                time|Query_Boolean|       Username|             Content|
+-----+----------+--------------------+-------------+---------------+--------------------+
|    0|1467810369|Mon Apr 06 22:19:...|     NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|    0|1467810672|Mon Apr 06 22:19:...|     NO_QUERY|  scotthamilton|is upset that he ...|
|    0|1467810917|Mon Apr 06 22:19:...|     NO_QUERY|       mattycus|@Kenichan I dived...|
|    0|1467811184|Mon Apr 06 22:19:...|     NO_QUERY|        ElleCTF|my whole body fee...|
|    0|1467811193|Mon Apr 06 22:19:...|     NO_QUERY|         Karoli|@nationwideclass ...|
|    0|1467811372|Mon Apr 06 22:20:...|     NO_QUERY|       joy_wolf|@Kwesidei not the...|
|    0|1467811592|Mon Apr 06 22:20:...|     NO_QUERY|        mybirch|         Need a hug |
|    0|1467811594|Mon Apr 06 22:20:...|     NO_QUERY|           coZZ|@LOLTrish hey  lo...|

In [61]:
df.createOrReplaceTempView("twitter")
result = spark.sql("""SELECT Username
                        ,Content
                   FROM twitter 
                   WHERE label = 0 
                   AND Content LIKE '%sad%'""")
result.show(5, False)

+--------------+--------------------------------------------------------------------------------------------------------------+
|Username      |Content                                                                                                       |
+--------------+--------------------------------------------------------------------------------------------------------------+
|gagoo         |im sad now  Miss.Lilly                                                                                        |
|kennypham     |Sad, sad, sad. I don't know why but I hate this feeling  I wanna sleep and I still can't!                     |
|DdubsShellBell|@JonathanRKnight Awww I soo wish I was there to see you finally comfortable! Im sad that I missed it          |
|hpfangirl94   |Falling asleep. Just heard about that Tracy girl's body being found. How sad  My heart breaks for that family.|
|peacoats      |is strangely sad about LiLo and SamRo breaking up.                                      

In [62]:
from pyspark.sql import functions as F

agg_df = df.groupBy("label").agg(F.count("id"))
agg_df.show()

+-----+---------+
|label|count(id)|
+-----+---------+
|    0|   800000|
|    4|   800000|
+-----+---------+



In [64]:
agg_df2 = df.groupBy("Query_Boolean").agg(F.count("id"))
agg_df2.show()

+-------------+---------+
|Query_Boolean|count(id)|
+-------------+---------+
|     NO_QUERY|  1600000|
+-------------+---------+

